<a href="https://colab.research.google.com/github/dspriggs-ds/general-notebooks/blob/main/azure_pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Azure Price Data Pipeline

## Overview

The following pipeline retrieves Azure Pricing, by Region,  data from a Microsoft API. The data is then saved to a Delta Lake table and exported to an Excel spreadsheet. The following technologies are used:

*   [Delta Lake](https://docs.microsoft.com/en-us/azure/databricks/delta/)
*   [Python](https://www.python.org/)
*   [Pandas](https://pandas.pydata.org/) 
*   [Jupyter Notebook](https://jupyter.org/)

## Process

The data pipeline connects to the [Microsoft Pricing API](https://docs.microsoft.com/en-us/rest/api/cost-management/retail-prices/azure-retail-prices#api-property-details) and retrieves the pricing data iteratively by region (see Overview for Azure regions). Next, the data is written to a Delta Lake table for storage. Last, the data is exported from the Delta Lake to an Excel workbook.

## Reference
https://github.com/santhoshkanala/azureretailpricesapi

### Mount Google Drive for file storage

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Download then install Java and Spark

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz
!tar xf spark-3.0.3-bin-hadoop3.2.tgz

In [ ]:
!pip -q install findspark

In [ ]:
import findspark
findspark.init()

### Set environment variables to use Spark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:0.7.0 --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog pyspark-shell'

### Import Packages

In [ ]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import *
from pyspark.sql.types import *
import json
import pandas as pd
import requests

### Instantiate Spark Session

In [ ]:
spark = SparkSession.builder.appName('delta_session').getOrCreate()
sc = spark.sparkContext

### Load Microsoft Azure pricing data for the US East, US East 2, US West and US West 2 Regions into JSON list

In [ ]:
regions = ["eastus","eastus2","westus","westus2"]
azurejslist = []

for region in regions:

  azureurl = "https://prices.azure.com/api/retail/prices?$filter=armRegionName%20eq%20%27{0}%27".format(region)
  response = requests.get(azureurl)

  while response.json()["NextPageLink"] != None:
    for i in response.json()['Items']:
          azurejslist.append(i)
    response = requests.get(response.json()["NextPageLink"]) 

### Load JSON List into Spark Dataframe 

In [ ]:
azureprice_df = spark.createDataFrame(azurejslist)

/content/spark-3.0.3-bin-hadoop3.2/python/pyspark/sql/session.py:381: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


### Write Spark Dataframe to Delta Lake for storage

In [ ]:
azureprice_df.write.format("delta").option("mergeSchema", "true").mode("append").save("/content/drive/MyDrive/delta/azureprice")

### Read data from Delta Lake and export to Excel File

In [ ]:
az_price_df = spark.read.format("delta").load("/content/drive/MyDrive/delta/azureprice").selectExpr("productName as Product","skuName as SubProduct","serviceName as Product_Type","location as Location","retailPrice as Price", "unitOfMeasure as Price_Unit_Of_Measure")

# az_price_df.toPandas().to_csv("/content/drive/MyDrive/output/azureprices.csv",index=False)
az_price_df.toPandas().to_excel("/content/drive/MyDrive/output/azureprices.xlsx",sheet_name="azure_prices",index=False)

### Clean up

In [ ]:
!rm -rf /content/spark-3.0.3-bin-hadoop3.2